In [388]:
 import math 
from sympy import*
from sympy.abc import*

def F_z(y):
    H = 0.014
    z = y / (math.sqrt(2) * H)
    F_z = (2 / math.sqrt(math.pi)) * (integrate(exp(-(x ** 2)), (x, 0, z)).evalf())
    return F_z

def dolya(x1, x2, y1, y2):
    ras = (1/4) * (F_z(x2) - F_z(x1)) * (F_z(y2) - F_z(y1))
    return ras

def dolya_prist(x1, x2, y1):
    ras = (1/4) * (F_z(x2) - F_z(x1)) * (1 - F_z(y1))
    return ras

def okrug(x):
    x = x * 10000
    ost = x % 1
    if ost >= 0.5:
        x = (x // 1) + 1
    else: 
        x = x // 1
    if x == 0:
        x = 1
    x /= 10000
    return x

def nax_fi(alfa):
    fi = 0.1
    t = math.tan(alfa * math.pi / 180)
    A_fi = ((1 - fi) * (2 ** 0.5)) / (fi ** 1.5)
    mu_fi = fi * ((fi / (2 - fi)) ** 0.5)
    t1 = (2 * A_fi * mu_fi) / ((((1 + ((1 - fi) ** 0.5)
                                 ) ** 2) - (4 * ((mu_fi * A_fi) ** 2))) ** 0.5)
    ta = (t * 1000) // 1
    ta1 = (t1 * 1000) // 1
    while ta != ta1:
        t = math.tan(alfa * math.pi / 180)
        A_fi = ((1 - fi) * (2 ** 0.5)) / (fi ** 1.5)
        mu_fi = fi * ((fi / (2 - fi)) ** 0.5)
        t1 = (2 * A_fi * mu_fi) / ((((1 + ((1 - fi) ** 0.5)) ** 2) -
                                    (4 * ((mu_fi * A_fi) ** 2))) ** 0.5)
        fi += 0.00001
        ta = (t * 1000) // 1
        ta1 = (t1 * 1000) // 1
    return fi

## Исходные данные

$P_п = 100 кН;~ p_к = 5 МПа;~p_a = 0.1 МПа \\Компоненты~топлива ~~~~ O_2 + H_2$

In [389]:
P_pust = 100000
p_k = 5 * 10 ** 6
p_a = 0.1 * 10**6

## Определение основных парраметров

### в ядре потока

$\alpha_{опт.я} = 0.54;~ K_0 = 7.937;~ K_m = 4.286;~ I_уп = 4045.88~\frac{м}{c}; \\ T_я = 3049.59K; R= 790 \frac{Дж}{кг \cdot К};~ k= 1.272$

In [390]:
alfa_optim_yadr = 0.54
K_0 = 7.937
K_m = alfa_optim_yadr * K_0
I_u_yadr = 4045.88
T_yadr = 3049.59
R = 790
k = 1.272

### В пристенеке 

$ T_{прист} = 2000 К;~ \alpha_{прист} = 0.28;~ K_{mприст} = 2.222;~ I_{уприст} = 3882$

In [391]:
T_prist = 2000
alfa_prist = 0.28
K_m_prist = alfa_prist * K_0
I_u_prist = 3882

### Параметры с учетом избытка горючего в пристенке
$ I_{уд} = 4037.686~ \frac{м}{с};~~ \dot m_{\Sigma} = 24.7~ \frac{кг}{с}$

In [392]:
I_ud = I_u_yadr - (0.05 * (I_u_yadr - I_u_prist))
m_sum = P_pust / I_ud

#### Расходы в пристенке 

$ \dot m_{прист} = 1.238~\frac{кг}{с};~ \dot m_{гприст} = 0.384~\frac{кг}{с}~ \dot m_{окприст} = 0.854~\frac{кг}{с}$

In [393]:
m_prist = m_sum * 0.05
m_g_prist = m_prist / (K_m_prist + 1)
m_ok_prist = m_prist - m_g_prist


#### Расходы в ядре 

$ \dot m_{я} = 23.52 \frac{кг}{с};~ \dot m_{гя} = 4.45 \frac{кг}{с};~ \dot m_{окя} = 19.08 \frac{кг}{с}$

In [394]:
m_yadr = m_sum - m_prist
m_g_yadr = m_yadr / (K_m + 1)
m_ok_yadr = m_yadr - m_g_yadr

#### Расходный комплекс

$ A_k = 0.662;~~ \beta = 2344.1\frac{м}{с}$

In [395]:
A_k = (k ** 0.5) * ((2 / (k + 1)) ** ((k + 1)  / (2 * (k - 1))))
betta = ((T_yadr * R) ** 0.5) / A_k

#### Параметры критического сечения 

$F_{кр} = 0.0116 м^2;~ D_{кр} = 0.121 м$

In [396]:
F_kr = (m_sum * betta) / p_k
D_kr = ((4 * F_kr) / 3.14) ** 0.5

#### Параметры форсуночной головки
##### Находим радиус форсуночной головкии и задаемся шагом между форсунками

$F_г = 0.046 м^2;~ D_г = 0.243 м;~ R_г = 0.121м;~ H = 0.014м;~ D_ш = 0.208м;$

In [397]:
F_g = 4 * F_kr
D_g = ((4 * F_g) / 3.14) ** 0.5
R_g = D_g / 2
H = 0.014
D_sh = D_g - (2.5 * H)

##### Определяем радиус расположения пристеночных форсунок 

$R_{прист} = 115 мм;$

In [398]:
R_prist = (D_g - H) / 2

#### Задаемся количеством форсунок в ядре и пристенке 
$n_{гпр} = 108;~ n_{окпр} = 36;~ n_{гя} = 65;~ n_{окпр} = 138;~$

In [399]:
n_g_prist = 108
n_ok_prist = 36
n_g_yadr = 61
n_ok_yadr =142 

#### Определеим соответствующие расходы через форсунки 

$\dot m_{фгя} = 0.0467 \frac{кг}{с};~ \dot m_{фокя} = 0.127 \frac{кг}{с};~\dot m_{фгприст} = 0.003 \frac{кг}{с};~\dot m_{я} = 0.005 \frac{кг}{с};~$

In [400]:
m_fg_yadr = m_g_yadr / n_g_yadr
m_fok_yadr = m_ok_yadr / n_ok_yadr
m_fg_prist = m_g_prist / n_g_prist
m_fok_prist = m_ok_prist / n_ok_prist

### Геометрический метод расчета соотношения компонентов в ядре потока

#### Для площадки с форсункой горючего по центру и форсунками окислителя по краям квадрата справедливо

$K_{km~геом} = 3.682$

In [401]:
n_o = 6 # Количество форсунок окислителя
m_otn_ok = 1 / 3 # Относительный расход каждой форсунки окислителя на площадку
n_g = 1 # Количество форсунок окислителя 
m_otn_g = 1 # Относительный расход каждой форсунки горючего на площадку
k_m_yadr_geom = (n_o * m_otn_ok * m_fok_yadr) / (n_g * m_otn_g * m_fg_yadr)

### Расчет соотношения компонентов методом Иевлева

#### Приняв что ддля каждого элемента принимабт участие форсунки, которые находятся на растояни 3-х шагов 

##### Площадка №1 (центральная)
$K_{m1} = 3.93$

In [402]:
XX1ok = [(-0.010, 0.010), (0.00212, 0.02212)]
YY1ok = [(0.004, 0.024), (-0.017, 0.003)]
d_sum = 0
for i1 in range(len(XX1ok)):
    d = dolya(XX1ok[i1][0], XX1ok[i1][1], YY1ok[i1][0], YY1ok[i1][1])
    d_sum += 6*d
m_ok1 = m_fok_yadr * d_sum

XX1g = [(-0.010, 0.010), (0.00212, 0.02212), (0.00212, 0.02212), (0.00212, 0.02212)]
YY1g = [(-0.010,0.010), (0.011, 0.031), (0.011, 0.031), (0.011, 0.031)]
d_sum = 0
for i1 in range(len(XX1g)):
    d = dolya(XX1g[i1][0], XX1g[i1][1], YY1g[i1][0], YY1g[i1][1])
    d_sum +=  2*d
m_g1 = m_fg_yadr * d_sum
k_m1 = m_ok1 / m_g1

#### Площадки №2, 3, 4 по радиуссу:
#### Распределение аналогично распределению площади №1, ввиду той же геометрии расположения форсунок в рассматриваемой зоне 3Н

#### Площадка №5 по радиусу:
$K_{m5} = 2.66$

In [403]:
XX5ok_yadr = [(-0.010, 0.010), (0.00212, 0.02212)]
YY5ok_yadr = [(0.004, 0.024), (-0.017, 0.003)]
d_sum_yadr = 0

for i5 in range(len(XX5ok_yadr)):
    d = dolya(XX5ok_yadr[i5][0], XX5ok_yadr[i5][1], YY5ok_yadr[i5][0], YY5ok_yadr[i5][1])
    d_sum_yadr += 4 * d
m_ok5_yadr = m_fok_yadr * d_sum_yadr

In [404]:
XX5ok_prist = [(0.004, 0.0236)]
YY5ok_prist = [(0.000325, 0.01967)]
d_sum_prist = 0

for i5 in range(len(XX5ok_prist)):
    d = dolya(XX5ok_prist[i5][0], XX5ok_prist[i5][1], YY5ok_prist[i5][0], YY5ok_prist[i5][1])
    d_sum_prist += 4 * d
m_ok5_prist = m_fok_prist * d_sum_prist
m_ok5 = m_ok5_prist + m_ok5_yadr

In [405]:
XX5g_yadr = [(-0.010, 0.010), (0.00212, 0.02212)]
YY5g_yadr = [(-0.010,0.010), (0.011, 0.031)]
d_sum_yadr = 0

for i5 in range(len(XX5g_yadr)):
    d = dolya(XX5g_yadr[i5][0], XX5g_yadr[i5][1], YY5g_yadr[i5][0], YY5g_yadr[i5][1])
    d_sum_yadr += 3*d
m_g5_yadr = m_fg_yadr * d_sum_yadr

In [406]:
XX5g_prist = [(0.00955, 0.0029), (0.0045, 0.024)]
YY5g_prist = [(-0.02, -0.0015), (-0.01, 0.01)]
d_sum_prist = 0

for i5 in range(len(XX5g_prist)):
    d = dolya(XX5g_prist[i5][0], XX5g_prist[i5][1], YY5g_prist[i5][0], YY5g_prist[i5][1])
    d_sum_prist += 2*d
m_g5_prist = m_fg_prist * d_sum_prist
m_g5 = m_g5_prist + m_g5_yadr
k_m5 = m_ok5 / m_g5

#### Площадка № 6 (нижняя по пристеночному слою)
$K_{m6} = 1.99$

In [407]:
XX6ok_yadr = [(0.004, 0.024)]
YY6ok_yadr = [0.01]
d_sum_yadr = 0

for i6 in range(len(XX6ok_yadr)):
    d = dolya_prist(XX6ok_yadr[i6][0], XX6ok_yadr[i6][1], YY6ok_yadr[i6])
    d_sum_yadr += 2*d

In [408]:
XX6ok_prist = [(-0.01967, 0.000325)]
YY6ok_prist = [-0.00358]
d_sum_prist = 0

for i6 in range(len(XX6ok_prist)):
    d = dolya_prist(XX6ok_prist[i6][0], XX6ok_prist[i6][1], YY6ok_prist[i6])
    d_sum_prist += 2 * d
m_6ok = (d_sum_yadr * m_fok_yadr) + (d_sum_prist * m_fok_prist)

In [409]:
XX6g_yadr = [(-0.01, 0.01)]
YY6g_yadr = [0.01]
d_sum_yadr = 0

for i6 in range(len(XX6g_yadr)):
    d = dolya_prist(XX6g_yadr[i6][0], XX6g_yadr[i6][1], YY6g_yadr[i6])
    d_sum_yadr += d


In [410]:
XX6g_prist = [(-0.01, 0.01), (0.00015, 0.02015), (-0.01, 0.01), (0.00015, 0.02015)]
YY6g_prist = [-0.00358, -0.009, -0.00958, 0.009]
d_sum_prist = 0

for i6 in range(len(XX6g_prist)):
    d = dolya_prist(XX6g_prist[i6][0], XX6g_prist[i6][1], YY6g_prist[i6])
    d_sum_prist += 2*d
m_6g = (d_sum_yadr * m_fg_yadr) + (d_sum_prist * m_fg_prist)
km_6 = m_6ok / m_6g

#### Площадка № 7 (средняя по пристеночному слою)
$K_{m7} = 1.41$

In [411]:
XX7ok_yadr = [(0.00957, 0.02158)]
YY7ok_yadr = [0.00919]
d_sum_yadr = 0

for i7 in range(len(XX7ok_yadr)):
    d = dolya_prist(XX7ok_yadr[i7][0], XX7ok_yadr[i7][1], YY7ok_yadr[i7])
    d_sum_yadr += d
m_7ok_yadr = d_sum_yadr * m_fok_yadr

In [412]:
XX7ok_prist = [(-0.01, 0.01)]
YY7ok_prist = [-0.0045]
d_sum_prist = 0

for i7 in range(len(XX7ok_prist)):
    d = dolya_prist(XX7ok_prist[i7][0], XX7ok_prist[i7][1], YY7ok_prist[i7])
    d_sum_prist += d
m_7ok_prist = d_sum_prist * m_fok_prist
m_7ok = m_7ok_prist + m_7ok_yadr

In [413]:
XX7g_prist = [(-0.00039,0.01967), (-0.01967, 0.00039), (0.00019, 0.02015), 
              (-0.01, 0.01), (-0.02015 ,-0.00019)]
YY7g_prist = [-0.00407, -0.00407, -0.00955, -0.01, -0.00955]
d_sum_prist = 0

for i7 in range(len(XX7g_prist)):
    d = dolya_prist(XX7g_prist[i7][0], XX7g_prist[i7][1], YY7g_prist[i7])
    d_sum_prist += d
m_7g_prist = d_sum_prist * m_fg_prist
m_7g = m_7g_prist + m_7g_yadr
km_7 = m_7ok / m_7g

#### Площадка № 8 (верхняя по пристеночному слою)
$K_{m8} = 0.96$

In [414]:
XX8ok_prist = [(-0.01967, 0.00048)]
YY8ok_prist = [-0.00408]
d_sum_prist = 0

for i8 in range(len(XX8ok_prist)):
    d = dolya_prist(XX8ok_prist[i8][0], XX8ok_prist[i8][1], YY8ok_prist[i8])
    d_sum_prist += 2 * d

m_8ok_prist = d_sum_prist * m_fok_prist
m_8ok = m_8ok_prist + m_8ok_yadr

In [415]:
XX8g_prist = [(-0.02927, -0.00912), (-0.03023, 0.01008), (-0.02927, -0.00912), (-0.03023, 0.01008), 
             (-0.01, 0.010), (-0.01, 0.01), (-0.02015, -0.00015), (0.00015, 0.02015)]
YY8g_prist = [-0.00281, -0.00823, -0.00283, -0.00823, -0.00425, -0.01, -0.00956, -0.00956]
d_sum_prist = 0

for i8 in range(len(XX8g_prist)):
    d = dolya_prist(XX8g_prist[i8][0], XX8g_prist[i8][1], YY8g_prist[i8])
    d_sum_prist += d
m_8g_prist = d_sum_prist * m_fg_prist
m_8g = m_8g_prist + m_8g_yadr
km_8 = m_8ok / m_8g

### Расчет центробежной форсунки окислителя в ядре

$\Delta p_ф = 2~МПа;~ \rho_{ок} = 1140 \frac{кг}{м^3};~ \eta_{ок} = 0.2 \cdot 10^{-3} Па \cdot с;~
\alpha = 45^{\circ}$



In [416]:
delta_p_f = 2 * 10 ** 6
ro_ok = 1140 
din_vyaz = 0.2 * (10 ** (-3))
alfa = 45

### Диаметр форсунки и толщина боковой стенки 
$D_ф = 12мм;~~ \delta_{ст}=2мм$

In [417]:
D_f = 0.012
delt = 0.002

### Димаетр камеры закручивания, входного тангенциального отверстия и число тангенциальных отверстий
$D_{кз}=8мм;~~d_{вх}=2мм;~~i_{вх}=4$

In [418]:
D_kz = D_f - (2 * delt)
d_vx = 0.002
r_vx = d_vx / 2
i_vx = 4

### Длина тангенциального отверстия 

$l_{вх}= 2.5мм$

In [419]:
l_vx = ((((D_f/2)**2)-((((D_kz-d_vx)/2))**2))**0.5)-((((D_kz/2)**2)-((((D_kz-d_vx)/2))**2))**0.5)

### Длина камеры закручивания, принимаем ${l'}_{кз} = 1$ 
$\\ l_{кз}=8мм$


In [420]:
l_sht_kz = 1
l_kz = l_sht_kz * D_kz

### угол наклона образующей конической поверхности перехода от камеры закручивания к соплу 
$\phi_c=45^\circ$

In [421]:
fi_c = 45

### Величина радиуса, на котором расположена ось входного тангеницального отверстия от оси форсунке
$R_{вх} = 3.5мм$

In [422]:
R_vx = (D_kz-d_vx) / 2

### Диаметр сопла форсунки 
$d_c = 5.6мм$

In [423]:
d_c = 0.7*D_kz

### Относительная длина сопла ${l'}_c=0.6$
### Длина сопла
$l_c=2.8мм$

In [425]:
l_shc=0.3
l_c = l_shc*d_c

### Угол между осью входного тангенциального отверстия и наппавлением оси форсунки $\theta = 90^{\circ}$

### Геометрическая характеристика центробежной форсунки
$A=2.1$

In [426]:
theta = 90
A = ((R_vx*(d_c/2)) / (i_vx*((d_vx/2)**2))) * math.sin(3.14*theta/180)

### Число Рейнольдса на входе в форсунку 
$Re_{вх} = 4.28 \cdot 10^5$

In [427]:
Re = (4 * m_fok_yadr) / (math.pi * din_vyaz * d_vx * math.sqrt(i_vx))

### Коэффициент трения при условиях на входе в форсунку
$\lambda = 0.183$

In [428]:
lambd = math.exp(((25.8) / ((math.log10(Re)) ** 2.58)) - 2)

### Эквивалентная геометрическая характеристика форсунки, учитывающая влияние вязкости реальной жидкости
$A_э = 2.5$

In [429]:
A_e = A / (1 + ((lambd/2) * ((((R_vx / (r_vx * i_vx)) ** 2) * (1 / i_vx)) - A)))

### Коэффициент живого сечения сопла форсунки 
$\varphi = 0.45$

In [430]:
fi = 1/ (((((A_e / math.sqrt(8)) + math.sqrt(((A_e ** 2)/8) - (1/27))) ** (1/3)) + \
     (((A_e / math.sqrt(8)) - math.sqrt(((A_e ** 2)/8) - (1/27))) ** (1/3))) ** 2)

### Коэффициент расхода в центробежной форсунки
$\mu = 0.242$

In [431]:
mu_fi = fi * ((fi / (2 - fi)) ** 0.5)

### Длина форсунки 
$h = 11мм;$

In [387]:
h = l_kz + l_c + (((D_kz - d_c) / 2) * math.tan(math.pi * fi_c / 180))